# Forecasting directional movements of stock-prices for intraday trading using LSTM and random-forest

In [30]:
import os
import random

import numpy as np
import pandas as pd

In [31]:
SEED = 9
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [32]:
SP500_df = pd.read_csv("data/SPXconst.csv")
all_companies = list(set(SP500_df.values.flatten()))
all_companies.remove(np.nan)
SP500_df.info()
SP500_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Columns: 348 entries, 01/1990 to 12/2018
dtypes: object(348)
memory usage: 1.3+ MB


,01/1990,02/1990,03/1990,04/1990,05/1990,06/1990,07/1990,08/1990,09/1990,10/1990,...,03/2018,04/2018,05/2018,06/2018,07/2018,08/2018,09/2018,10/2018,11/2018,12/2018
0,S626,S626,S626,S626,S626,S626,S626,S626,S626,S626,...,S456,S456,S456,S456,S456,S456,S456,S456,S456,S198
1,S1335,S1335,S1335,S490,S490,S490,S490,S490,S490,S490,...,S698,S309,S309,S309,S309,S309,S309,S76,S76,S550
2,S83,S83,S83,S1335,S1335,S1335,S1141,S1141,S1141,S1141,...,S709,S698,S698,S698,S30,S30,S30,S309,S309,S118
3,S1090,S1090,S1090,S83,S83,S83,S1335,S1335,S1335,S1335,...,S658,S976,S976,S976,S698,S698,S698,S976,S976,S1368
4,S59,S59,S59,S1090,S1090,S1090,S83,S83,S83,S83,...,S834,S709,S709,S709,S631,S631,S631,S709,S709,S1170


In [33]:
constituents = {
    "-".join(col.split("/")[::-1]): set(SP500_df[col].dropna())
    for col in SP500_df.columns
}
print(len(constituents))

348


In [34]:
constituents_train = {}
for test_year in range(1993, 2016):
    months = [
        str(t) + "-0" + str(m) if m < 10 else str(t) + "-" + str(m)
        for t in range(test_year - 3, test_year)
        for m in range(1, 13)
    ]
    constituents_train[test_year] = [list(constituents[m]) for m in months]
    constituents_train[test_year] = {
        i for sublist in constituents_train[test_year] for i in sublist
    }